In [4]:
!pip install streamlit pyngrok -qqq
!pip install --upgrade langchain_core langchain_community vllm langchain_huggingface langchain_chroma sentence_transformers chromadb pymupdf -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.8 MB/s eta 0:00:

In [14]:
%%writefile app.py
import streamlit as st
from langchain_community.llms import VLLM
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import fitz
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PersianPDFSplitter class
class PersianPDFSplitter:
    def __init__(
        self,
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        fall_back_splitter=RecursiveCharacterTextSplitter,
        fall_back_splitter_kwargs={},
    ):
        self.__len = length_function
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.fall_back_splitter = fall_back_splitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            **fall_back_splitter_kwargs,
        )

    def create_documents(self, pdf_addr):
        doc = fitz.open(pdf_addr)
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text("text")
        list_of_documents = []
        if self.__len(text) > self.chunk_size:
            for t in self.fall_back_splitter.split_text(text):
                list_of_documents.append(Document(page_content=t, metadata={"heading": "Full Text"}))
        else:
            list_of_documents.append(Document(page_content=text, metadata={"heading": "Full Text"}))
        return list_of_documents

# Cache model loading
@st.cache_resource
def load_model():
    model_id = "TitanML/ChatQA-1.5-8B-AWQ-4bit"
    llm = VLLM(
        model=model_id,
        trust_remote_code=True,
        max_new_tokens=128,
        top_k=10,
        top_p=0.95,
        temperature=0.8,
        vllm_kwargs={"quantization": "awq", "enforce_eager": True},
    )
    model_name = "mixedbread-ai/mxbai-embed-large-v1"
    model_kwargs = {'trust_remote_code': True}
    encode_kwargs = {'normalize_embeddings': False}
    hf = HuggingFaceEmbeddings(
        model_name=model_name,
        encode_kwargs=encode_kwargs,
        model_kwargs=model_kwargs
    )
    rsps = PersianPDFSplitter(chunk_size=1000, chunk_overlap=100)
    documents = rsps.create_documents("Drugs.pdf")
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=hf,
        collection_name="drugs_pdf"
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    system = "System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
    instruction = "Please give a full and complete answer in persian for the question."
    template = "<|begin_of_text|>" + system + "\n\n" + "{context}" + "\n\n" + "User: " + instruction + " " + "{question}" + "\n\nAssistant:"
    prompt = PromptTemplate.from_template(template)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents=False
    )
    return qa_chain

# Streamlit UI
st.title("سامانه پرس‌وجو درباره داروها")
st.write("سوال خود را درباره دیکلوفناک یا سایر داروها به فارسی وارد کنید:")
qa_chain = load_model()
question = st.text_input("سوال شما:", "")
if st.button("ارسال"):
    if question:
        with st.spinner("در حال پردازش..."):
            result = qa_chain.invoke({"query": question})
            st.write("**پاسخ:**")
            st.write(result["result"])
    else:
        st.warning("لطفاً یک سوال وارد کنید.")
st.write("**سوالات پیشنهادی:**")
example_questions = [
    "دیکلوفناک چیست و چه کاربردی دارد؟",
    "شیاف دیکلوفناک برای چه مواردی استفاده می‌شود؟",
    "نحوه عملکرد دیکلوفناک چگونه است؟",
    "دوز روزانه مجاز دیکلوفناک برای بزرگسالان چقدر است؟",
    "عوارض جانبی رایج دیکلوفناک کدامند؟"
]
for q in example_questions:
    if st.button(q):
        with st.spinner("در حال پردازش..."):
            result = qa_chain.invoke({"query": q})
            st.write("**پاسخ:**")
            st.write(result["result"])

Overwriting app.py


In [16]:
from google.colab import files
uploaded = files.upload()  # Upload Drugs.pdf

Saving Drugs.pdf to Drugs.pdf


In [18]:
from pyngrok import ngrok
import os
# Set your ngrok auth token
ngrok.set_auth_token("31jXVKaABcRrp4gmPiwFffkLXVr_3Mb4ow7xnEPwkHsuN1nUe")

# Start Streamlit server on port 8501
os.system("nohup streamlit run app.py --server.port 8501 &")

# Create a public URL with ngrok
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")

Streamlit app is running at: NgrokTunnel: "https://066eee5e863f.ngrok-free.app" -> "http://localhost:8501"
